### Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import scvelo as scv
import anndata as ad
import moscot
import cellrank as cr
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, facecolor='white')

In [ ]:
scv.set_figure_params()
adata_l= scv.read("data/10x/1_1/velocyto/1_1.loom")

In [ ]:
adata_l.obs_names = adata_l.obs_names.str.replace('1_1:', '', regex=True)

In [ ]:
adata_l.obs_names = adata_l.obs_names.str.replace('x', '', regex=True)

In [ ]:

# we will use this loom file to add the spliced/unspliced data obtained with velocyto to the gene counts obtained by cellranger
adata_cellranger = sc.read_10x_mtx("data/10x/1_1/outs/filtered_feature_bc_matrix/", var_names="gene_symbols", cache=False).copy()


In [ ]:
adata_cellranger.obs_names = adata_cellranger.obs_names.str.replace('-1', '', regex=True)

In [ ]:
adata_loom = scv.utils.merge(adata_cellranger,adata_l)

In [ ]:
# Change the index name in the obs DataFrame
adata_loom.obs.index.name = 'CellID'

In [ ]:
adata = adata_loom

In [ ]:
adata.obs

In [ ]:
adata.var_names_make_unique()

### Quality Control

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')

sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], log1p=True , inplace=True)

In [ ]:
variable_names = adata.var_names
mt_genes = [gene for gene in variable_names if gene.startswith('MT-')]
print(mt_genes)

In [ ]:
sc.pl.violin(adata, ["n_genes_by_counts", "total_counts",'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.n_genes_by_counts >1500, :]
adata = adata[adata.obs["pct_counts_mt"] < 20].copy()

In [ ]:
print(f"#cells after MT filter: {adata.n_obs}")

### Normalization

The next preprocessing step is normalization. A common approach is count depth scaling with subsequent log plus one (log1p) transformation. Count depth scaling normalizes the data to a “size factor” such as the median count depth in the dataset, ten thousand (CP10k) or one million (CPM, counts per million). The size factor for count depth scaling can be controlled via target_sum in pp.normalize_total. We are applying median count depth normalization with log1p transformation (AKA log1PF).

**target_sum=1e4:** This is the total count to which each cell's expression will be scaled. After normalization, the sum of all gene expression values in each cell will be equal to 10,000 (i.e., 1e4). This is a common choice in scRNA-seq normalization because it converts gene expression counts to a scale that is more manageable and easier to compare across cells.

* The raw counts of gene expression for each cell are divided by the total sum of counts in that cell.
* The result is then multiplied by target_sum=1e4, making the total expression for each cell equal to 10,000. This standardization removes differences caused by varying sequencing depth across cells.

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata, target_sum=1e4)
# Logarithmize the data
sc.pp.log1p(adata)

### Feature selection
As a next step, we want to reduce the dimensionality of the dataset and only include the most informative genes. This step is commonly known as feature selection. The scanpy function pp.highly_variable_genes annotates highly variable genes by reproducing the implementations of Seurat [Satija et al., 2015], Cell Ranger [Zheng et al., 2017], and Seurat v3 [Stuart et al., 2019] depending on the chosen flavor.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
#top10_genes = adata.var[adata.var['highly_variable']].sort_values(by='dispersions', ascending=False).head(10).index

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata.obs

### cell cycle scoring 

In [ ]:
# Assuming adata is your AnnData object

# Define cell cycle genes
cell_cycle_genes = {
    'G1/S': [
        'MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2', 'MCM6', 'CDCA7', 'DTL',
        'PRIM1', 'UHRF1', 'CENPU', 'HELLS', 'RFC2', 'RPA2', 'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP',
        'CCNE2', 'UBR7', 'POLD3', 'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN',
        'DSCC1', 'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8'
    ],
    'G2/M': [
        'HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80', 'CKAP2L', 'CKAP2', 'AURKB',
        'BUB1', 'KIF11', 'ANP32E', 'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDCA3', 'CDC20', 'TTK', 'CDC25C',
        'KIF2C', 'RANGAP1', 'NCAPG', 'DLGAP5', 'CDCA2', 'CDCA8', 'ECT2', 'KIF23', 'HMMR', 'AURKA', 'PSRC1',
        'ANLN', 'LBR', 'CKAP5', 'CENPF', 'CENPE', 'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA'
    ]
}

# Create dataframes for the cell cycle genes
s_genes = pd.Series(cell_cycle_genes['G1/S'], name='S')
g2m_genes = pd.Series(cell_cycle_genes['G2/M'], name='G2M')

# Score the cell cycle genes
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

# Visualize the cell cycle scores
sc.pl.scatter(adata, x='S_score', y='G2M_score')

# Visualize cell cycle phase assignments
#sc.pl.umap(adata, color=['S_score', 'G2M_score', 'phase'])



In [ ]:
# regressing out the Mitochondrial genes and cell cycle correction 
sc.pp.regress_out(adata, keys=['S_score', 'G2M_score', 'pct_counts_mt'])

### Scale Data

In [ ]:
sc.pp.scale(adata, max_value=10)

### Dimensionality Reduction
Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

### Nearest neighbor graph constuction and visualization


In [ ]:
sc.pp.neighbors(adata,n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.louvain(adata, flavor='vtraag', resolution=0.56)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=["louvain"])
sc.pl.umap(adata, color=["phase"])

In [ ]:
for res in [0.02, 0.56, 2.0]:
    sc.tl.louvain(
        adata, key_added=f"louvain_res_{res:4.2f}", resolution=res, flavor='vtraag'
    )

In [ ]:
sc.pl.umap(
    adata,
    color=["louvain_res_0.02", "louvain_res_0.56", "louvain_res_2.00"],
    
)

## RNA Velocity

In [ ]:
scv.pl.proportions(adata)
scv.pp.moments(adata, n_pcs=50, n_neighbors=10)
##scv.tl.recover_dynamics(adata, n_jobs=8)
#scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity(adata, mode='stochastic')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', color='louvain')

In [ ]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="louvain_res_0.56", method="wilcoxon")
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="louvain_res_0.56", standard_scale="var", n_genes=5
)

In [ ]:
#adding the meta-models the velocity 


pd.set_option("display.precision", 9)
import urllib.request

# Fetch the script from the URL
url = "https://raw.githubusercontent.com/kris-nader/sc-type-py/main/sctype_py.py"
response = urllib.request.urlopen(url)
script = response.read().decode()

# Execute the script
exec(script)

In [ ]:
adata.layers["counts"] = adata.X.copy()
scaled_data = pd.DataFrame(adata.X)
# change column indexes
scaled_data.columns =adata.var_names
# Change the row indexes
scaled_data.index = adata.obs_names
scaled_data=scaled_data.T

In [ ]:
adata.X

In [ ]:
scRNAseqData=scaled_data
gs_list=gene_sets_prepare(path_to_db_file="/beegfs/scratch/ric.tonon/ric.tonon/Hamed_plasticity/share/data/ScTypeDB_full.xlsx",cell_type="metamodules")
es_max = sctype_score(scRNAseqData = scRNAseqData, scaled = True, gs = gs_list['gs_positive'], gs2 = gs_list['gs_negative'])

unique_clusters = adata.obs['louvain_res_2.00'].unique()
# Apply the function to each unique cluster and combine the results into a DataFrame
cL_results = pd.concat([process_cluster(cluster,adata,es_max,'louvain_res_2.00') for cluster in unique_clusters])

# Group by cluster and select the top row based on scores
sctype_scores = cL_results.groupby('cluster').apply(lambda x: x.nlargest(1, 'scores')).reset_index(drop=True)

# Set low-confidence clusters to "Unknown"
sctype_scores.loc[sctype_scores['scores'] < sctype_scores['ncells'] / 4, 'type'] = 'Unknown'

# Iterate over unique clusters
adata.obs['sctype_classification'] = ""
for cluster in sctype_scores['cluster'].unique():
    # Filter sctype_scores for the current cluster
    cl_type = sctype_scores[sctype_scores['cluster'] == cluster]
    # Get the type for the current cluster
    cl_type_value = cl_type['type'].iloc[0]
    # Update 'sctype_classification' in pbmc.obs for cells belonging to the current cluster
    adata.obs.loc[adata.obs['louvain_res_2.00'] == cluster, 'sctype_classification'] = cl_type_value

# Plot the UMAP with sctype_classification as labels
sc.pl.umap(adata, color='sctype_classification', title='UMAP with sctype_classification')

In [ ]:
import pandas as pd

# Load the tab-delimited file into a pandas DataFrame
file_path = 'metamodules.txt'  # Replace with the correct file path
modules_df = pd.read_csv(file_path, delimiter='\t', index_col=0)

# Convert the DataFrame into a dictionary of lists
# Each column represents a module, and each row represents genes
modules_dict = {col: modules_df[col].dropna().tolist() for col in modules_df.columns}

# Check the module dictionary (optional)
print(modules_dict)

In [ ]:
import scanpy as sc

# Calculate the scores for each module
for module_name, gene_list in modules_dict.items():
    sc.tl.score_genes(adata, gene_list, score_name=module_name)

# Check the adata.obs to see the added module scores (optional)
print(adata.obs.head())

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

# List of module names (replace with actual names from your data)
module_names = list(modules_dict.keys())

# Create subplots: 1 row, N columns (side-by-side)
fig, axes = plt.subplots(1, len(module_names), figsize=(5 * len(module_names), 5))

# Plot each UMAP in a separate subplot
for i, module_name in enumerate(module_names):
    sc.pl.umap(adata, color=module_name, cmap='RdYlBu_r', size=40, ax=axes[i], show=False)
    axes[i].set_title(f'UMAP: {module_name}')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
sc.pl.umap(adata, color=["louvain_res_0.56", "log1p_total_counts", "pct_counts_mt", "log1p_n_genes_by_counts"])

In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('Pop_GeneLists2.csv')

# Group the genes based on the signature they belong to
# Assuming the signature names are unique in the 'List' column
signatures = df.groupby('List')['Name'].apply(list).to_dict()

# You will now have a dictionary where the keys are the signature names
# and the values are the corresponding lists of genes

In [ ]:
# Calculate scores for each signature
for signature_name, gene_list in signatures.items():
    # Calculate the score for each gene signature
    sc.tl.score_genes(adata, gene_list, score_name=signature_name + '_score')

# Now the scores will be stored in adata.obs with column names like 'Signature1_score', 'Signature2_score', etc.

In [ ]:
# Visualize the signature scores on UMAP
sc.pl.umap(adata, color=[signature_name + '_score' for signature_name in signatures.keys()],size=100, cmap='RdYlBu_r')

### Cell Rank

#### velocity kernel

In [ ]:
vk = cr.kernels.VelocityKernel(adata)

In [ ]:
vk

In [ ]:
vk.compute_transition_matrix()

In [ ]:
vk.plot_projection(color = 'louvain_res_0.56')

## finding initial & terminal states with velocity kernel

In [ ]:
g = cr.estimators.GPCCA(vk)
print(g)

In [ ]:
g.fit(cluster_key="louvain_res_0.56", n_states=[2, 10])
g.plot_macrostates(which="all", discrete=True, legend_loc="right", s=100)

In [ ]:
g.predict_terminal_states()
g.plot_macrostates(which="terminal", legend_loc="right", s=100)

In [ ]:
g.predict_initial_states(allow_overlap=True)
g.plot_macrostates(which="initial", legend_loc="right", s=100)

In [ ]:
g

In [ ]:
g.plot_coarse_T()

#### Pseudotime (dpt) kernel

In [ ]:
# I repeat removing cluster6
adata_copy = adata[adata.obs['louvain'] != '6'].copy()

In [ ]:
sc.tl.diffmap(adata_copy)

In [ ]:
adata_copy.obsm['X_diffmap'][:, 3].argmax()

In [ ]:
root_ixs = 1902  # has been found using `adata.obsm['X_diffmap'][:, 3].argmax()`

scv.pl.scatter(
    adata_copy,
    basis="diffmap",
    c=["louvain_res_0.56", root_ixs],
    legend_loc="right",
    components=["2, 4"],
)

adata_copy.uns["iroot"] = root_ixs

In [ ]:
sc.tl.dpt(adata_copy)
sc.pl.embedding(
    adata_copy,
    basis="umap",
    color=["dpt_pseudotime"],
    color_map="gnuplot2",
)


In [ ]:
pk = cr.kernels.PseudotimeKernel(adata_copy, time_key="dpt_pseudotime")
pk.compute_transition_matrix()

print(pk)

In [ ]:
#following plot are purely informed by the pseudotime and the umap graph, and not by RNA velocity
pk.plot_projection(basis="umap", recompute=True)

## finding initial & terminal states with pseudotime(dpt) kernel

In [ ]:
g3 = cr.estimators.GPCCA(pk)
print(g3)

In [ ]:
g3.fit(cluster_key="louvain_res_0.56", n_states=[2, 12])
g3.plot_macrostates(which="all", discrete=True, legend_loc="right", s=100)

In [ ]:
g3.predict_terminal_states()
g3.plot_macrostates(which="terminal", legend_loc="right", s=100)

In [ ]:
g3.predict_initial_states(allow_overlap=True)
g3.plot_macrostates(which="initial", legend_loc="right", s=100)

In [ ]:
g3.set_terminal_states(states=["1_1, 1_2", "3", "4","0"])
g3.plot_macrostates(which="terminal", legend_loc="right", s=100)

In [ ]:
g3.plot_coarse_T()

## Fate Probabilities

for the moment we chose the pseudotime kernel(basic) to go on with

In [ ]:
g3.compute_fate_probabilities()
g3.plot_fate_probabilities(same_plot=False)

In [ ]:
g3.plot_fate_probabilities(same_plot=True)

In [ ]:
fev_states = [ "1", "3", "0","4"]
sc.pl.embedding(
    adata_copy, basis="umap", color="louvain_res_0.56", groups=fev_states, legend_loc="right"
)

In [ ]:
cr.pl.circular_projection(adata_copy, keys=["louvain_res_0.56"], legend_loc="right")

In [ ]:
cr.pl.aggregate_fate_probabilities(
    adata_copy,
    mode="violin",
    lineages=["0"],
    cluster_key="louvain_res_0.56",
    clusters=fev_states,
)

In [ ]:
driver_clusters = [ "1", "3", "0","4"]

zero_df = g3.compute_lineage_drivers(
    lineages=["0"], cluster_key="louvain_res_0.56", clusters=driver_clusters)
#six_df.to_csv ('../time_course_hamed/genes/term_8.csv', index = True, header=True)

In [ ]:
# Filter for correlation over 0.3 and sort by Pval in ascending order
filtered_sorted_0_df = zero_df[zero_df["0_corr"] > 0.3].sort_values(by="0_pval")


filtered_sorted_0_df.to_csv ('filterd_term_0_cl.csv', index = True, header=True)

filtered_sorted_0_df.head(20)

In [ ]:
zero_df.head(14)

In [ ]:
driver_clusters = ["1", "3", "4","0"]

one_df = g3.compute_lineage_drivers(
    lineages=["1_1, 1_2"], cluster_key="louvain_res_0.56", clusters=driver_clusters
)

#one_df.to_csv ('../cellrank/term_genes/term_1_with.csv', index = True, header=True)

In [ ]:
# Filter for correlation over 0.3 and sort by Pval in ascending order
filtered_sorted_1_df = one_df[one_df["1_1, 1_2_corr"] > 0.3].sort_values(by="1_1, 1_2_pval")


filtered_sorted_1_df.to_csv ('filterd_term_1_cl.csv', index = True, header=True)

filtered_sorted_1_df.head(20)

In [ ]:
one_df.head(27)

In [ ]:
driver_clusters = ["1", "3", "4","0"]

three_df = g3.compute_lineage_drivers(
    lineages=["3"], cluster_key="louvain_res_0.56", clusters=driver_clusters
)

#three_df.to_csv ('../cellrank/term_genes/term_3_with.csv', index = True, header=True)

In [ ]:
# Filter for correlation over 0.3 and sort by Pval in ascending order
filtered_sorted_3_df = three_df[three_df["3_corr"] > 0.3].sort_values(by="3_pval")


filtered_sorted_3_df.to_csv ('filterd_term_3_cl.csv', index = True, header=True)

filtered_sorted_3_df.head(20)

In [ ]:
three_df.head(17)

In [ ]:
driver_clusters = ["1", "3", "4","0"]

four_df = g3.compute_lineage_drivers(
    lineages=["4"], cluster_key="louvain_res_0.56", clusters=driver_clusters
)

#four_df.to_csv ('../cellrank/term_genes/term_4_with.csv', index = True, header=True)

In [ ]:
# Filter for correlation over 0.3 and sort by Pval in ascending order
filtered_sorted_4_df = four_df[four_df["4_corr"] > 0.3].sort_values(by="4_pval")


filtered_sorted_4_df.to_csv ('filterd_term_4_cl.csv', index = True, header=True)

filtered_sorted_4_df.head(20)

In [ ]:
four_df.head(10)

In [ ]:
adata_copy.obs["fate_probabilities_1"] = g3.fate_probabilities["1_1, 1_2"].X.flatten()

sc.pl.embedding(
    adata_copy,
    basis="umap",
    color=["fate_probabilities_1"] + list(one_df.index[:11]),
    color_map="viridis",
    s=50,
    ncols=3,
    vmax="p96",
)

In [ ]:
adata_copy.obs["fate_probabilities_3"] = g3.fate_probabilities["3"].X.flatten()

sc.pl.embedding(
    adata_copy,
    basis="umap",
    color=["fate_probabilities_3"] + list(three_df.index[:11]),
    color_map="viridis",
    s=50,
    ncols=3,
    vmax="p96",
)

In [ ]:
adata_copy.obs["fate_probabilities_4"] = g3.fate_probabilities["4"].X.flatten()

sc.pl.embedding(
    adata_copy,
    basis="umap",
    color=["fate_probabilities_4"] + list(four_df.index[:11]),
    color_map="viridis",
    s=50,
    ncols=3,
    vmax="p96",
)

In [ ]:
adata_copy.obs["fate_probabilities_0"] = g3.fate_probabilities["0"].X.flatten()

sc.pl.embedding(
    adata_copy,
    basis="umap",
    color=["fate_probabilities_0"] + list(zero_df.index[:11]),
    color_map="viridis",
    s=50,
    ncols=3,
    vmax="p96",
)

In [ ]:
model = cr.models.GAM(adata_copy, distribution='gaussian', link= 'identity')

In [ ]:
# plot heatmap
cr.pl.heatmap(
    adata_copy,
    model=model,  # use the model from before
    lineages="0",
    cluster_key="louvain_res_0.56",
    show_fate_probabilities=True,
    genes=zero_df.head(37).index,
    time_key="dpt_pseudotime",
    figsize=(12, 10),
    show_all_genes=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
# plot heatmap
cr.pl.heatmap(
    adata_copy,
    model=model,  # use the model from before
    lineages="1_1, 1_2",
    cluster_key="louvain_res_0.56",
    show_fate_probabilities=True,
    genes=one_df.head(37).index,
    time_key="dpt_pseudotime",
    figsize=(12, 10),
    show_all_genes=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
# plot heatmap
cr.pl.heatmap(
    adata_copy,
    model=model,  # use the model from before
    lineages="3",
    cluster_key="louvain_res_0.56",
    show_fate_probabilities=True,
    genes=three_df.head(37).index,
    time_key="dpt_pseudotime",
    figsize=(12, 10),
    show_all_genes=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
# plot heatmap
cr.pl.heatmap(
    adata_copy,
    model=model,  # use the model from before
    lineages="4",
    cluster_key="louvain_res_0.56",
    show_fate_probabilities=True,
    genes=four_df.head(37).index,
    time_key="dpt_pseudotime",
    figsize=(12, 10),
    show_all_genes=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
# Filter for correlation over 0.3 and sort by Pval in ascending order
filtered_sorted_6_df = six_df[six_df["6_corr"] > 0.3].sort_values(by="6_pval")


filtered_sorted_6_df.to_csv ('../time_course_hamed/genes/filterd_term_6_cl.csv', index = True, header=True)

filtered_sorted_6_df.head(20)

In [ ]:
cr.pl.gene_trends(
    adata_filtered,
    model=model,
    genes=["PTPRZ1", "BCAN", "C1orf61", "MALAT1", "KCNQ1OT1", "DST", "PCDH9", "PTN", "DSEL", "RFX4", "REV3L", "CCND2", "POU3F2", "NCAN"],
    same_plot=True,
    ncols=3,
    time_key="dpt_pseudotime",
    hide_cells=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Load the gene signature file
gene_signatures = pd.read_csv('metamodules.txt',sep="\t")

# Define the gene sets for each meta-module
# MES2    MES1    AC      OPC     NPC1_sig        NPC2_sig        G1/S    G2/M
gene_sets = {
    'MES1': gene_signatures['MES1'].dropna().tolist(),
    'MES2': gene_signatures['MES2'].dropna().tolist(),
    'NPC1': gene_signatures['NPC1_sig'].dropna().tolist(),
    'NPC2': gene_signatures['NPC2_sig'].dropna().tolist(),
    'AC': gene_signatures['AC'].dropna().tolist(),
    'OPC': gene_signatures['OPC'].dropna().tolist(),
    'G1S': gene_signatures['G1/S'].dropna().tolist(),
    'G2M': gene_signatures['G2/M'].dropna().tolist()
}


var_names_list =adata_copy.var_names.tolist()

In [ ]:
adata_copy

In [ ]:
# Calculate single-cell gene signature scores
def calculate_signature_scores(adata_copy, gene_sets):
    scores = {}
    for key, genes in gene_sets.items():
        # Filter genes to include only those present in adata.var_names
        valid_genes = [gene for gene in genes if gene in var_names_list]
        
        if not valid_genes:
            continue
        
        # Calculate average relative expression for the valid gene set
        avg_expr = adata_copy[:, valid_genes].X.mean(axis=1)
        
        # Create control gene set
        control_genes = []
        for gene in valid_genes:
            expr_level = adata_copy[:, gene].X.mean()
            bin_genes_bool = np.ravel((adata_copy.X.mean(axis=0) > expr_level - 0.05) & (adata_copy.X.mean(axis=0) < expr_level + 0.05))
            bin_genes = np.array(var_names_list)[np.array(bin_genes_bool)]
            sample_size = min(100, len(bin_genes))
            control_genes.extend(np.random.choice(bin_genes, sample_size, replace=False))
            #control_genes.extend(np.random.choice(bin_genes, 100, replace=True))
        
        # Calculate average relative expression for the control gene set
        avg_control_expr = adata_copy[:, control_genes].X.mean(axis=1)
        
        # Calculate signature score
        scores[key] = avg_expr - avg_control_expr
    
    return scores


In [ ]:
signature_scores = calculate_signature_scores(adata_copy, gene_sets)



In [ ]:
signature_scores

In [ ]:
# Assign cells to meta-modules and their hybrids
def assign_meta_modules(signature_scores):
    meta_modules = ['MES', 'NPC', 'AC', 'OPC']
    assignments = []
    both_scores = []
    for i in range(len(signature_scores['MES1'])):
        scores = {key: signature_scores[key][i] for key in signature_scores}
        # Collapse MES1 and MES2 into MES, and NPC1 and NPC2 into NPC
        # Create a new dictionary to store the updated keys
        # Create a new dictionary to store the updated keys
        updated_scores = {}
        
        # Combine values for MES and NPC keys
        mes_total = 0
        npc_total = 0
        
        for key, value in scores.items():
            if key in ['MES1', 'MES2']:
                mes_total += value
            elif key in ['NPC1', 'NPC2']:
                npc_total += value
            else:
                updated_scores[key] = value
        
        # Add the combined MES and NPC values to the new dictionary
        updated_scores['MES'] = mes_total
        updated_scores['NPC'] = npc_total
        scores = updated_scores
        # Assign to the highest scoring meta-module
        primary_module = max(scores, key=scores.get)
        # Check for hybrid assignment
        hybrid_module = None
        second_highest_score = sorted(scores.values(), reverse=True)[1]
        
        #if second_highest_score > 1 or second_highest_score > np.percentile([scores[key] for key in meta_modules if key != primary_module], 10) or (second_highest_score - sorted(scores.values(), reverse=True)[2]) >= 0.3:
         #   hybrid_module = [key for key in scores if scores[key] == second_highest_score][0]
        #if second_highest_score > 1:
         #   hybrid_module = [key for key in scores if scores[key] == second_highest_score][0]

        # Higher 0.3
        if second_highest_score - sorted(scores.values(), reverse=True)[2] >= 0.3:
            hybrid_module = [key for key in scores if scores[key] == second_highest_score][0]
        # percentile
        #if second_highest_score > np.percentile([scores[key] for key in meta_modules if key != primary_module], 10):
            #hybrid_module = [key for key in scores if scores[key] == second_highest_score][0]
            
        assignments.append((primary_module, hybrid_module))
        both_scores.append((scores, second_highest_score))
    
    return assignments,both_scores
    

In [ ]:
assignments,both_scores = assign_meta_modules(signature_scores)

In [ ]:
# Produce a plot of expected number of hybrids
def plot_expected_hybrids(assignments):
    hybrid_counts = {module: 0 for module in ['MES', 'NPC', 'AC', 'OPC','G1S','G2M']}
    
    for primary, hybrid in assignments:
        if hybrid:
            hybrid_counts[primary] += 1
    
    expected_hybrid_counts = {module: [] for module in ['MES', 'NPC', 'AC', 'OPC','G1S','G2M']}
    
    for _ in range(100):
        shuffled_assignments = np.random.permutation(assignments)
        shuffled_hybrid_counts = {module: 0 for module in ['MES', 'NPC', 'AC', 'OPC','G1S','G2M']}
        
        for primary, hybrid in shuffled_assignments:
            if hybrid:
                shuffled_hybrid_counts[primary] += 1
        
        for module in shuffled_hybrid_counts:
            expected_hybrid_counts[module].append(shuffled_hybrid_counts[module])
    
    means = {module: np.mean(expected_hybrid_counts[module]) for module in expected_hybrid_counts}
    stds = {module: np.std(expected_hybrid_counts[module]) for module in expected_hybrid_counts}
    
    fig, ax = plt.subplots()
    
    ax.bar(hybrid_counts.keys(), hybrid_counts.values(), label='Observed')
    ax.errorbar(means.keys(), means.values(), yerr=stds.values(), fmt='o', label='Expected')
    
    ax.set_xlabel('Meta-module')
    ax.set_ylabel('Number of hybrids')
    ax.legend()
    
    plt.show()



In [ ]:
plot_expected_hybrids(assignments)

In [ ]:
hybrid_list = assignments
shuffled_assignments = np.random.permutation(assignments)

# Filter the list to include only hybrid combinations
hybrids = [tuple(sorted(item)) for item in hybrid_list if item[1] != None]

# Count the occurrences of each hybrid combination
hybrid_counts = Counter(hybrids)

# Calculate the total number of elements in the list
total_elements = len(hybrid_list)

# Calculate the percentage of each hybrid combination
hybrid_percentages = {k: (v / total_elements) * 100 for k, v in hybrid_counts.items()}

# Convert tuple keys to strings for plotting
hybrid_percentages_str = {f"{k},{k}": v for k, v in hybrid_percentages.items()}

# Create a bar plot
fig, ax = plt.subplots(figsize=(12, 8))
ax.bar(hybrid_percentages_str.keys(), hybrid_percentages_str.values())

# Set labels and title
ax.set_xlabel('Hybrid Combinations')
ax.set_ylabel('Percentage (%)')
ax.set_title('Percentage of Hybrid Combinations')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
expected_hybrid_counts = {module: [] for module in ['MES', 'NPC', 'AC', 'OPC','G1S','G2M']}
for _ in range(100):
    shuffled_assignments = np.random.permutation(assignments)
    shuffled_hybrid_counts = {module: 0 for module in ['MES', 'NPC', 'AC', 'OPC','G1S','G2M']}
    
    for primary, hybrid in shuffled_assignments:
        if hybrid:
            shuffled_hybrid_counts[primary] += 1
    
    for module in shuffled_hybrid_counts:
        expected_hybrid_counts[module].append(shuffled_hybrid_counts[module])

means = {module: np.mean(expected_hybrid_counts[module]) for module in expected_hybrid_counts}
stds = {module: np.std(expected_hybrid_counts[module]) for module in expected_hybrid_counts}


In [ ]:
means
stds

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter


hybrid_list = assignments
shuffled_assignments = np.random.permutation(assignments)

    
def process_hybrid_list(hybrid_list):
    # Filter the list to include only hybrid combinations and sort the tuples
    filtered_list = [tuple(sorted(item)) for item in hybrid_list if item[1] != None]
    
    # Count the occurrences of each hybrid combination
    hybrid_counts = Counter(filtered_list)
    
    # Calculate the total number of elements in the list
    total_elements = len(hybrid_list)
    
    # Calculate the percentage of each hybrid combination
    hybrid_percentages = {k: (v / total_elements) * 100 for k, v in hybrid_counts.items()}
    
    return hybrid_percentages

# Process both lists
percentages_list = process_hybrid_list(hybrid_list)
percentages_shuffles = process_hybrid_list(shuffled_assignments)

# Convert tuple keys to strings for plotting
percentages_list_str = {f"{k},{k}": v for k, v in percentages_list.items()}
percentages_shuffles_str = {f"{k},{k}": v for k, v in percentages_shuffles.items()}

# Create a bar plot
fig, ax = plt.subplots(figsize=(12, 8))

# Define the width of the bars
bar_width = 0.35

# Define the positions of the bars
index = range(len(percentages_list_str))

# Plot the bars for the first list
bars1 = ax.bar(index, percentages_list_str.values(), bar_width, label='Observed', color='blue')

# Plot the bars for the second list, offset by bar_width
bars2 = ax.bar([i + bar_width for i in index], percentages_shuffles_str.values(), bar_width, label='Expected', color='orange')

# Set labels and title
ax.set_xlabel('Hybrid Combinations')
ax.set_ylabel('Percentage (%)')
ax.set_title('Percentage of Hybrid Combinations')

# Set the x-ticks to be in the middle of the grouped bars
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(percentages_list_str.keys(), rotation=45, ha='right')

# Add a legend
ax.legend()


# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
shuffled_assignments = np.random.permutation(assignments)
shuffled_assignments

In [ ]:
hybrid_list = [
    ('MES', 'NPC'),
    ('NPC', 'None'),
    ('NPC', 'None'),
    ('MES', 'None'),
    ('NPC', 'None'),
    ('G1S', 'MES'),
    ('AC', 'MES'),
    ('NPC', 'None'),
    ('NPC', 'None'),
    ('MES', 'NPC'),
    ('MES', 'None'),
    # Add more elements as needed
]

# Filter the list to include only hybrid combinations
hybrids = [item for item in hybrid_list if item != 'None']

# Count the occurrences of each hybrid combination
hybrid_counts = Counter(hybrids)

In [ ]:
filtered_hybrids = [item for item in hybrid_list if item[1] != 'None']
filtered_hybrids = [item for item in hybrid_list if item[1] != 'None']
print(filtered_hybrids )

In [ ]:
hybrid_list = [
    ('MES', 'NPC'),
    ('NPC', 'None'),
    ('NPC', 'None'),
    ('MES', 'None'),
    ('NPC', 'None'),
    ('G1S', 'MES'),
    ('AC', 'MES'),
    ('NPC', 'None'),
    ('NPC', 'None'),
    ('MES', 'NPC'),
    ('MES', 'None'),
]

filtered_list = [item for item in hybrid_list if item[1] != 'None']
print(filtered_list)

In [ ]:
# Create a new dictionary to store the updated keys
updated_signature_scores = {}

# Calculate the element-wise maximum for MES and NPC
mes_max = np.maximum(signature_scores.get('MES1', np.zeros_like(signature_scores['MES1'])), signature_scores.get('MES2', np.zeros_like(signature_scores['MES2'])))
npc_max = np.maximum(signature_scores.get('NPC1', np.zeros_like(signature_scores['NPC1'])), signature_scores.get('NPC2', np.zeros_like(signature_scores['NPC2'])))

# Add the new MES and NPC to the updated dictionary
updated_signature_scores['MES'] = mes_max
updated_signature_scores['NPC'] = npc_max

# Add the other keys to the updated dictionary
for key, value in signature_scores.items():
    if key not in ['MES1', 'MES2', 'NPC1', 'NPC2']:
        updated_signature_scores[key] = value



In [ ]:
updated_signature_scores



In [ ]:
def generate_2d_representation(signature_scores1):
    D_values = []
    x_values = []
    
    for i in range(len(signature_scores1['MES'])):
        SCopc_npc = max(signature_scores1['OPC'][i], signature_scores1['NPC'][i])
        SCac_mes = max(signature_scores1['AC'][i], signature_scores1['MES'][i])
        
        D = SCopc_npc - SCac_mes
        D_values.append(D)
        
        if D > 0:
            x_diff = signature_scores1['OPC'][i] - signature_scores1['NPC'][i]
        else:
            x_diff = signature_scores1['AC'][i] - signature_scores1['MES'][i]
        
        # Apply log2 to the absolute value and keep the sign
        x_values.append(np.sign(x_diff) * np.log2(abs(x_diff) + 1))
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    ax.scatter(x_values, D_values)
    
    ax.set_xlabel('log2(|SCopc – SCnpc|+1) or log2(|SCac–SCmes|)')
    ax.set_ylabel('D value')
    
    # Set the plot axes to be y -1,1 and x -1,1
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])

    # Add dashed black lines at x=0 and y=0
    ax.axhline(0, color='black', linestyle='--')
    ax.axvline(0, color='black', linestyle='--')

    # Add text labels to the edges of the four regions
    ax.text(-0.9, 0.9, 'NPC-like', fontsize=12, ha='center', va='center')
    ax.text(0.9, 0.9, 'OPC-like', fontsize=12, ha='center', va='center')
    ax.text(-0.9, -0.9, 'MES-like', fontsize=12, ha='center', va='center')
    ax.text(0.9, -0.9, 'AC-like', fontsize=12, ha='center', va='center')
    
    plt.show()


In [ ]:
def generate_2d_representation(signature_scores1, adata_copy, color_by, distance = 0.25):
    D_values = []
    x_values = []
    
    for i in range(len(signature_scores1['MES'])):
        SCopc_npc = max(signature_scores1['OPC'][i], signature_scores1['NPC'][i])
        SCac_mes = max(signature_scores1['AC'][i], signature_scores1['MES'][i])
        
        D = SCopc_npc - SCac_mes
        D_values.append(D)
        
        if D > 0:
            x_diff = signature_scores1['NPC'][i] - signature_scores1['OPC'][i]
        else:
            x_diff = signature_scores1['MES'][i] - signature_scores1['AC'][i]
        
        # Apply log2 to the absolute value and keep the sign
        x_values.append(np.sign(x_diff) * np.log2(abs(x_diff) + 1))
    
    # Convert lists to numpy arrays
    D_values = np.array(D_values)
    x_values = np.array(x_values)
    
    # Identify cells in the center of the plot
    center_mask = (D_values >= -1*distance) & (D_values <= distance) & (x_values >= -1*distance) & (x_values <= distance)
    center_cells = adata_copy.obs[center_mask]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    if pd.api.types.is_numeric_dtype(adata_copy.obs[color_by]):
        # Use a continuous palette for numerical data
        scatter = ax.scatter(x_values, D_values, c=adata_copy.obs[color_by], cmap='viridis', alpha=0.7)
        plt.colorbar(scatter, ax=ax, label=color_by)
    else:
        # Extract the color palette used by Scanpy for categorical data
        sc.pl.umap(adata_copy, color=color_by, show=False)
        cluster_colors = adata_copy.uns[f'{color_by}_colors']
        
        # Map categorical data to colors
        color_map = {category: color for category, color in zip(adata_copy.obs[color_by].cat.categories, cluster_colors)}
        colors = adata_copy.obs[color_by].map(color_map)
        
        scatter = ax.scatter(x_values, D_values, c=colors, alpha=0.7)
    
    # Highlight center cells
    #ax.scatter(x_values[center_mask], D_values[center_mask], edgecolor='red', facecolor='none', s=100, label='Center Cells')
    
    ax.set_xlabel('log2(|SCopc – SCnpc|+1) or log2(|SCac–SCmes|)')
    ax.set_ylabel('D value')
    
    # Set the plot axes to be y -1,1 and x -1,1
    ax.set_xlim([-4, 4])
    ax.set_ylim([-4, 4])

    # Add dashed black lines at x=0 and y=0
    ax.axhline(0, color='black', linestyle='--')
    ax.axvline(0, color='black', linestyle='--')

    # Add text labels to the edges of the four regions
    ax.text(-2.7, 2.7, 'OPC-like', fontsize=12, ha='center', va='center')
    ax.text(2.7, 2.7, 'NPC-like', fontsize=12, ha='center', va='center')
    ax.text(-2.7, -2.7, 'AC-like', fontsize=12, ha='center', va='center')
    ax.text(2.7, -2.7, 'MES-like', fontsize=12, ha='center', va='center')
    
    ax.legend()
    plt.show()
    
    return center_cells


center_cells = generate_2d_representation(updated_signature_scores, adata_copy, color_by='louvain_res_0.56',
                                         distance=0.10)
#print("Center cells:", center_cells)


In [ ]:
# Highlight center cells on UMAP plot
adata_copy.obs['center_cells'] = 'Other'
adata_copy.obs.loc[center_cells.index, 'center_cells'] = 'Center'
sc.pl.umap(adata_copy, color='center_cells', groups=['Center'], palette=['red', 'lightgrey'])


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='phase')

In [ ]:
def calculate_centroids(signature_scores1, adata_copy, cluster_key):
    D_values = []
    x_values = []
    
    for i in range(len(signature_scores1['MES'])):
        SCopc_npc = max(signature_scores1['OPC'][i], signature_scores1['NPC'][i])
        SCac_mes = max(signature_scores1['AC'][i], signature_scores1['MES'][i])
        
        D = SCopc_npc - SCac_mes
        D_values.append(D)
        
        if D > 0:
            x_diff = signature_scores1['OPC'][i] - signature_scores1['NPC'][i]
        else:
            x_diff = signature_scores1['AC'][i] - signature_scores1['MES'][i]
        
        # Apply log2 to the absolute value and keep the sign
        x_values.append(np.sign(x_diff) * np.log2(abs(x_diff) + 1))
    

    # Convert lists to numpy arrays and flatten them
    D_values = np.array(D_values).flatten()
    x_values = np.array(x_values).flatten()
    
    # Ensure the lengths match
    if len(x_values) != len(adata_copy.obs[cluster_key].values) or len(D_values) != len(adata_copy.obs[cluster_key].values):
        raise ValueError("The lengths of x_values, D_values, and cluster labels must match.")
    # Create a DataFrame with the cluster labels and the calculated X, Y values
    df = pd.DataFrame({
        'cluster': adata_copy.obs[cluster_key].values,
        'X': x_values,
        'Y': D_values
    })
    
    # Calculate the centroid for each cluster
    centroids = df.groupby('cluster').mean()
    
    return centroids


centroids = calculate_centroids(updated_signature_scores, adata_copy, cluster_key='louvain_res_0.56')
print("Centroids for each cluster:")
print(centroids)


In [ ]:
def plot_centroids(centroids):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot the centroids
    ax.scatter(centroids['X'], centroids['Y'], color='blue', s=200)
    
    # Annotate the centroids with cluster labels
    for cluster, row in centroids.iterrows():
        ax.text(row['X'], row['Y'], cluster, fontsize=12, ha='center', va='center', color='red')
    
    ax.set_xlabel('log2(|SCopc – SCnpc|+1) or log2(|SCac–SCmes|)')
    ax.set_ylabel('D value')
    
    # Set the plot axes to be y -1,1 and x -1,1
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])

    # Add dashed black lines at x=0 and y=0
    ax.axhline(0, color='black', linestyle='--')
    ax.axvline(0, color='black', linestyle='--')

    plt.show()


plot_centroids(centroids)


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='MES1')


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='AC')


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='OPC')


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='NPC1_sig')


In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='NPC2_sig')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='G1/S')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='G2/M')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='Pop1_score')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='Pop2_score')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='Pop3_score')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='Pop4_score')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='section')

In [ ]:
generate_2d_representation(updated_signature_scores, adata_copy, color_by='plasticity_status')